Option chains
=======

In [1]:
%gui qt5

from ib_insync import *
util.useQt()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=12)

Suppose we want to find the options on the SPY. There are two ways to do that:
* The old way by requesting contract details
* The new and faster way

So first the old way. It starts with an ambiguous Option contract and uses that
as a wildcard to get the details of all contracts that match:

** This will take a while **

In [2]:
option = Option('SPY', exchange='SMART')
cds = ib.reqContractDetails(option)

contracts = [cd.summary for cd in cds]

print(len(contracts))
contracts[0]

4638


Contract(conId=178534485, symbol='SPY', secType='OPT', lastTradeDateOrContractMonth='20171215', strike=100.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   171215C00100000', tradingClass='SPY')

So that's a few thousand contracts. Let's put in some restrictions to get this number down:

* Use only the first 3 expirations after today that are on a Friday
* Use only strike prices within +- 20 dollar of the current SPY market price
* Use only strike prices that are a multitude of 5 dollar

For the first restriction the expirations are filtered with an isFriday method,
made unique with set(), then sorted and finally the first 3 taken:

In [3]:
import datetime

def isFriday(date):
    y = int(date[0:4])
    m = int(date[4:6])
    d = int(date[6:8])
    dd = datetime.date(y, m, d)
    return dd.weekday() == 4 and dd > datetime.date.today()

expirations = sorted(set(c.lastTradeDateOrContractMonth for c in contracts
            if isFriday(c.lastTradeDateOrContractMonth)))[:3]

expirations

['20170714', '20170721', '20170728']

Hmmm... perhaps we could have just taken the next three Fridays?
But the number of contracts is going down nicely:

In [4]:
contracts = [c for c in contracts if c.lastTradeDateOrContractMonth in expirations]

print(len(contracts))

572


To get the current price, first create the SPY contract:

In [5]:
spy = Stock('SPY', 'ARCA')

ib.qualifyContracts(spy)

[Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')]

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [6]:
[ticker] = ib.reqTickers(spy)

ticker

Ticker(contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), time=datetime.datetime(2017, 7, 10, 22, 25, 55, 779317), bid=243.13, bidSize=42, ask=243.15, askSize=114, last=243.16, lastSize=3, volume=66338, high=243.33, low=242.14, close=0.0, ticks='[...]')

Apply the final two restrictions:

In [7]:
spyPrice = ticker.marketPrice()

contracts = [c for c in contracts if
        spyPrice - 20 < c.strike < spyPrice + 20 and
        c.strike % 5 == 0]

print(len(contracts))
print(contracts[0])

oldContracts = contracts  # remember for later

48
Contract(conId=270350402, symbol='SPY', secType='OPT', lastTradeDateOrContractMonth='20170721', strike=225.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170721C00225000', tradingClass='SPY')


Finally we have a list of usable option contracts.

Okay so now the new and faster way:

In [16]:
chains = ib.reqSecDefOptParams(spy.symbol, '', spy.secType, spy.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NASDAQOM,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
1,PSE,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
2,BATS,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
3,AMEX,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
4,PEARL,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
5,NASDAQBX,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
6,ISE,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
7,MIAX,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
8,SMART,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."
9,CBOE,756733,SPY,100,"{20170929, 20191220, 20171229, 20170712, 20170...","{224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, ..."


We want the options that trade on SMART:

In [9]:
chain = next(c for c in chains if c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId=756733, tradingClass='SPY', multiplier='100', expirations={'20170929', '20191220', '20171229', '20170712', '20170825', '20170804', '20180119', '20170811', '20171215', '20171020', '20170915', '20180629', '20180316', '20190315', '20180720', '20170719', '20170728', '20190621', '20180615', '20180921', '20170721', '20190118', '20170809', '20170802', '20181221', '20170726', '20170818', '20170714', '20180329'}, strikes={224.5, 10.0, 15.0, 20.0, 232.5, 245.5, 25.0, 30.0, 225.5, 35.0, 228.5, 40.0, 45.0, 246.5, 50.0, 55.0, 226.5, 60.0, 65.0, 70.0, 247.5, 218.5, 75.0, 219.5, 80.0, 220.5, 227.5, 85.0, 221.5, 90.0, 222.5, 95.0, 248.5, 223.5, 249.5, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 14

What we have here is a matrix of expirations x strikes. From this we can build all the contracts:

In [15]:
strikes = [strike for strike in chain.strikes if
        strike % 5 == 0 and
        spyPrice - 20 < strike < spyPrice + 20]
expirations = sorted(exp for exp in chain.expirations if isFriday(exp))[:3]
rights = ['P', 'C']

contracts = [Option('SPY', expiration, strike, right, 'SMART')
        for right in rights for expiration in expirations for strike in strikes]

ib.qualifyContracts(*contracts)

print(len(contracts))
print()
print(contracts[0])

48

Option(conId=278038391, symbol='SPY', lastTradeDateOrContractMonth='20170714', strike=225.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170714P00225000', tradingClass='SPY')


Let's see if the new way ends up with the same contracts as the old way:

In [11]:
set(contracts) == set(oldContracts)

True

Yep. Now to get the market data for all options in one go:

In [12]:
tickers = ib.reqTickers(*contracts)

tickers

[Ticker(contract=Option(conId=278038391, symbol='SPY', lastTradeDateOrContractMonth='20170714', strike=225.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170714P00225000', tradingClass='SPY'), time=datetime.datetime(2017, 7, 10, 22, 26, 13, 339731), bid=0.45, bidSize=9526, ask=0.48, askSize=18077, close=0.0, ticks='[...]'),
 Ticker(contract=Option(conId=278038402, symbol='SPY', lastTradeDateOrContractMonth='20170714', strike=230.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170714P00230000', tradingClass='SPY'), time=datetime.datetime(2017, 7, 10, 22, 26, 13, 339731), bid=0.57, bidSize=556, ask=0.59, askSize=712, last=0.58, lastSize=4, close=0.0, ticks='[...]'),
 Ticker(contract=Option(conId=278038442, symbol='SPY', lastTradeDateOrContractMonth='20170714', strike=235.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   170714P00235000', tradingClass='SPY'), time=datetime.datetim

Let's ask IB what their model thinks the implied volatility is for one option, given the current market price:

In [13]:
t = tickers[0]

ib.calculateImpliedVolatility(t.contract, t.marketPrice(), spyPrice)

OptionComputation(tickType=53, impliedVol=0.514995942258687, delta=None, optPrice=0.46499999999999997, pvDividend=None, gamma=None, vega=None, theta=None, undPrice=243.16)

Sometimes there is no reply and the request will time out. Try a different ticker then.

Let's now do the reverse: Assume the volatility is 0.2, what does IB think the option price should be?

In [14]:
ib.calculateOptionPrice(t.contract, 0.2, spyPrice)

OptionComputation(tickType=53, impliedVol=0.2, delta=-0.0001178861294523798, optPrice=0.000151702466871658, pvDividend=None, gamma=8.787553264478968e-05, vega=0.0001863353947806617, theta=None, undPrice=243.16)